# Introduction

The time derivative of magnetization can be expressed as follows:

$$\frac{\partial \mathbf{m} }{ \partial t} = \boldsymbol{\tau}$$

where $\mathbf{m}$ is the reduced magnetization ($\mathbf{m} = \mathbf{M}/M_\text{S}$), and $\boldsymbol{\tau}$ describes torques acting on magnetization. In mumax3 are implemented three basic torques:

* Landau-Lifshitz torque, $\boldsymbol{\tau}_{\text{LL}}$
* Zhang-Li spin-transfer torque
* Slonczewski spin-transfer torque

During this labs, we will focus only on Landau-Lifshitz torque:

$$\boldsymbol{\tau}_{\text{LL}} = \gamma _\text{LL} \frac{1}{1+\alpha^2} \left( \mathbf{m} \times \mathbf{B}_\text{eff} + \alpha \mathbf{m} \times \left( \mathbf{m} \times \mathbf{B}_\text{eff}  \right) \right)$$

where $\mathbf{B}_\text{eff}$ is the effective magnetic field consisting from:

* externally aplied field (Zeeman field)  $\mathbf{B}_\text{ext}$, (in mumax: ` B_ext `),
* Heisenberg exchange field  $\mathbf{B}_\text{exch}$ , (in mumax: ` B_exch `),
* dipolar field  $\mathbf{B}_\text{demag}$, (in mumax: ` B_demag `),
* Dzyaloshinskii-Moriya exchange field  $\mathbf{B}_\text{dmi}$, (in mumax: ` B_dm `),
* magnetocrystalline anisotropy field  $\mathbf{B}_\text{anis}$, (in mumax: ` B_anis `),
* thermal field  $\mathbf{B}_\text{therm}$, (in mumax: ` B_therm `).


See details in "The design and verification of mumax3", AIP Advances 4, 107133 (2014):
http://scitation.aip.org/content/aip/journal/adva/4/10/10.1063/1.4899186

# How to prepare a simulation
Simulation scripts should be written in simplified `go` language.
Very good tutorials can be found on the [mumax3 examples page](http://mumax.github.io/examples.html)
## Steps of modeling:
### Static and dynamic simulations:
1.  Definition of the mesh (discretization), i.e., lateral dimensions of the system under study, cell size, periodic boundary conditions if needed (with the number of repetitions)
2.  Definition of the geometry
3.  Definition of regions that can have different magnetic parameters
4.  Definition of magnetic parameters ($M_\text{S}$, $A_\text{ex}$, $\alpha$ ....) which then be matched to particular regions
5.  Definition of the initial magnetization configuration (if we do not specify it, mumax will start from the random magnetic configuration), example.: `m = uniform(0.1, 1.0, 0)`
6.  Definition of the external magnetic field
7.  System relaxation/stabilization, functions `relax()` or `minimize()`
--------------------------
### Dynamic simulations only:
8.  Selection of solver and discretization in time
9.  Definition of the sampling (frequency of the data saving)
10. dynamic simulations( function: `run(float how_long)` or `RunWhile(bool condition)`)



### How to run a simulation

Scripts with simulations must have `.mx3` extension (`myfile.mx3`)!

To run a simulation, right-click a `.mx3` file and select `run code` or open the file and click on the arrow in the top right corner of the editor. Don't forget to save it before!

The output is automatically stored in the `myfile.zarr` directory. 

To view the simulation webui as it is running, open (`https://amumax.x.zfn2.matmoa.xyz`)

## Simulations of a static magnetic configuration 


### Relaxation 

To relax the system you can use command: `relax()` or `minimize()`. 


Important: sometimes, an error can occur when simulations are initiated from an ideally saturated configuration, e.g. `m = uniform(0, 0, 1)`. It is a bug. In order to avoid such an issue, better to set an almost ideally saturated system: `m = uniform(1e-7, 0, 1)`. 


### Data storing 


#### PNG files -- `Snapshot(q Quantity, fname string)` 


To save data you can use `Snapshot(m)`, `Snapshot(m.comp(2))` or `saveas(m, "m_stab")`. The last option give you a full information about the magnetic configuration whereas `Snapshot` function generate png files. 


#### Zarr files -- `SaveAs(q Quantity, fname string)`, `AutoSave(q Quantity, period float64)` 


In order to save some quantity from simulations (`m, m.comp(), B_ext, B_demag, ...`) one may use `SaveAs` function saving the vector (or scalar) fields as a zarr array: 


`SaveAs(q Quantity, fname string)` 


for example: 


`SaveAs(B_ext.comp(2), "B_ext_z")` 


Sometimes it would be more usefull to **generate filenames automatically**. For instance, one can introduce to the name value of some variables, e.g., `B_ext`: 


` saveas(m,Sprintf("m_field%3.4f",B_ext)) ` 


In the case of dynamic simulations, it is more convenient to save data with a precise sampling interval. In order to do so, we can use: 


`AutoSave(q Quantity, period float64)` 


for instance: 


`AutoSave(m, 2e-11)` 


that will autosave magnetization (all three components) with the interval of 20 ps. It gives a resolution up to 25 GHz in the spectral domain. 

### Task 1: Run the simulation

Calculate static magnetic configuration in a 1000 nm long and 200 nm wide and 5 nm thick permalloy stripe.
The system has the following magnetic parameters: 

* the magnetization saturation $M_S=800 \times 10^3$ A/m, 
* the exchange constant $A_\mathrm{ex} = 13 \times 10^{-12}$ J/m, 
* the damping constant $\alpha=0.001$.

Use the following discretization: $d_x\times d_y \times d_z = 2 \times 2 \times 5~\mathrm{nm}^3$.

Save results of each simulation as a Zarr array and a PNG file using the commands:
* `save(m) // saving as Zarr array`
* `snapshot(m) // saving as PNG`

In the file `lab1/simulations/task1.mx3`, modify it so that mumax calculates the stable magnetization of the system for 4 different field values:

a) $B_\mathrm{ext} = (0, 0, 0)$, i.e., `B_ext=vector(0, 0, 0)`,

b) $B_\mathrm{ext} = (0, 1~\mathrm{T}, 0)$, i.e., `B_ext=vector(0, 1, 0)`,

c) $B_\mathrm{ext} = (0, 0, 0.5~\mathrm{T})$, i.e., `B_ext=vector(0, 0, 0.5)`,

d) $B_\mathrm{ext} = (0, 0, 1~\mathrm{T})$, i.e., `B_ext=vector(0, 0, 1)`.

For each $B_\mathrm{ext}$ you will relax the system and save the magnetization as an image and an array with the name `m_a` for a), `m_b` for b) and so on.

Don't hesitate to refer to the [mumax API](https://mumax.github.io/api.html)

### Working with `.zarr` data using the `pyzfn` library
Source code of the library: [https://github.com/MathieuMoalic/pyzfn](https://github.com/MathieuMoalic/pyzfn)

In [ ]:
from pyzfn import Pyzfn as op           # 'op' function to open the .zarr files
from pyzfn import utils                 # other various utilities
import numpy as np                      # working with vector data
from matplotlib import pyplot as plt    # plotting graphs
import cmocean                          # pretty colormaps
utils.load_mpl_style()                  # pretty graphs

In [ ]:
# Open the simulation data in python
task1 = op("./simulations/task1.zarr")
# To visualize which arrays you saved in a simulation:
task1.p

In [ ]:
# Getting simulation metadata:
# json file located at ./simulations/p1.zarr/.attrs
# You can access it like a python dict:
dx = task1.attrs["dx"] 
# or as a class attribute (given that you have no dataset named similarly):
dx = task1.dx
print(f"{dx=} m")

In [ ]:
# Get data you saved in amumax:
#             t,z,y,x,c
mag = task1.m[0,0,:,:,0] # `mag` is a numpy array
print(f"{mag.shape=}")

In [ ]:
# Some very useful info
task1.m.info

### A simple visualization example

In [ ]:
plt.figure() # create empty figure
#                  t,z,y,x,c
plt.imshow(task1.m[0,0,:,:,1])

### A more thourough visualization:
Use the previous code to visualize the in-plane magnetization along the y axis for a),b),c) and d)

In [ ]:
def plot_2d_magnetization(path:str, dset:str):
    task = op(path) # Open the simulation data in python
    fig, axes = plt.subplots(3,1, figsize=(10, 5), sharex=True, sharey=True) # create empty figure
    for comp,ax in enumerate(axes): # loop over the 3 components of the magnetization
        im = ax.imshow( # plot the magnetization component
            task[dset][0,0,:,:,comp],  # `mag` is a numpy array
            cmap=cmocean.cm.balance,  # pretty colormap
            origin="lower", # put the origin at the bottom left
            extent=[0, task.Tx*1e9, 0, task.Ty*1e9], # set the x and y axis limits ( in nm )
            vmin=-1, # set the color scale limits ( reducded magnetization is from -1 to 1 )
            vmax=1 # set the color scale limits ( reducded magnetization is from -1 to 1 )
            )
        ax.grid(True) # show the grid
        cb = fig.colorbar(im, ax=ax) # add a colorbar
        cb.set_label(f"m_{['x','y','z'][comp]}") # set the colorbar label
        ax.set_ylabel("y (nm)") # set the y axis label
    axes[-1].set_xlabel("x (nm)") # set the x axis label
    plt.tight_layout() # make the plot pretty


plot_2d_magnetization(path="./simulations/task1.zarr", dset="m_a") # call the function